# Lars Fuel -- A General Piecewise Linear Modeling Example

Suppose the Lars family has expanded their moisture farming business to start producing fuel for the podraces on Tatooine.

Here are the relevant data and details:

* They produce gas 1 and gas 2, from oil 1 and oil 2
* Gas 1 must contain at least 50% oil 1, gas 2  at least 60% 
* Gas 1 sells for \\$2.12/gallon, gas 2 for \\$2.44/gallon
* Current available: 300 gallons of oil 1, 200 gallons of oil 2 
* To purchase oil 1: \\$2.75/gallon for first 500, \\$2.00/gallon next 500, \\$1.80/gallon up to 1500 gallons
* To purchase oil 2: \\$2.20/gallon for first 300, \\$1.50/gallon next 300, up to 2300 gallon


How much of each oil should be purchased, and how should the oils be mixed to maximize the Lars family’s profit?


In [1]:
using JuMP, Gurobi

fuels = [:1, :2]
oils = [:1, :2]

profit = Dict(zip(fuels,[2.12, 2.44]))
# lower bound on percent of oil 1 in each gas
lb_o1 = Dict(zip(fuels, [0.5, 0.6]))

avail = Dict(zip(oils,[300, 200]))

# piecewise linear function data for f1
a10 = 0; a11 = 500; a12 = 1000; a13 = 1500
m11 = 2.75; m12 = 2; m13 = 1.8
b11 = 0; b12 = 500*2.75; b13 = 500*2.75 + 500*2

# piecewise linear function data for f2
a20 = 0; a21 = 300; a22 = 2300
m21 = 2.2; m22 = 1.5
b21 = 0; b22 = 300*2.2

m = Model(Gurobi.Optimizer)
set_optimizer_attribute(m, "OutputFlag",0)

@variable(m, v[oils] >= 0) # gallons of each oil to buy
@variable(m, x[oils,fuels] >= 0) # gallons of oil i that goes into fuel j
@variable(m, z[fuels] >= 0) # gallons of each fuel to make
@variable(m, y1[1:3], Bin) # binary variable selecting the interval for f1
@variable(m, y2[1:2], Bin) # binary variable selecting the interval for f2
@variable(m, w1[1:3] >= 0) # variable to represent v[1]
@variable(m, w2[1:2] >= 0) # variable to represent v[2]
@variable(m, t[1:2] >= 0) # varaible that will replace f1 and f2 in the objective

@objective(m, Max, sum(profit[i]*z[i] for i in fuels) - t[1] - t[2])

# use available oil
@constraint(m, availability[i in oils], sum(x[i,j] for j in fuels) <= avail[i] + v[i])
# relate gas made to x varaibles
@constraint(m, iden[j in fuels], sum(x[i,j] for i in oils) == z[j])
# lower bound on composition
@constraint(m, lb[j in fuels], x[:1,j] >= lb_o1[j]*z[j])

# relate t vars to cost functions
@constraint(m, t[1] == m11*w1[1] + b11*y1[1] + m12*w1[2] + b12*y1[2] + m13*w1[3] + b13*y1[3])
@constraint(m, t[2] == m21*w2[1] + b21*y2[1] + m22*w2[2] + b22*y2[2])

# bounds on the w variables
@constraints(m, 
begin
        a10*y1[1] <= w1[1]
        w1[1] <= a11*y1[1]
        a11*y1[2] <= w1[2] 
        w1[2] <= a12*y1[2]
        a12*y1[3] <= w1[3] 
        w1[3] <= a13*y1[3]
        end
)
@constraints(m, 
begin
        a20*y2[1] <= w2[1]
        w2[1] <= a21*y2[1]
        a21*y2[2] <= w2[2] 
        w2[2] <= a22*y2[2]
        end
)

# select only one segment (one y var = 1)
@constraint(m, sum(y1) == 1)
@constraint(m, sum(y2) == 1)

# set w vars equal to v vars
@constraint(m, sum(w1) == v[:1])
@constraint(m, sum(w2) == v[:2])

optimize!(m)

println("Purchase ", value(v[1]), " gallons of oil 1 and ", value(v[2]), " gallons of oil 2 to make ", 
    value(z[1]), " gallons of fuel 1 and ", value(z[2]), " gallons of fuel 2.")

Academic license - for non-commercial use only - expires 2022-06-27
Purchase 0.0 gallons of oil 1 and 0.0 gallons of oil 2 to make 0.0 gallons of fuel 1 and 500.0 gallons of fuel 2.
